# REHEATFUNQ Quickstart
This notebook can be used as a template to quickly investigate regional
aggregate heat flow distributions and fault-generated heat flow anomalies.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from reheatfunq.regional import default_prior, HeatFlowPredictive
from reheatfunq.anomaly import HeatFlowAnomalyPosterior, AnomalyLS1980

Here load the heat flow data and the fault trace:

In [ ]:
import json
with open('data/example-heatflow.json','r') as f:
    data = json.load(f)

fault_trace = np.array(list(zip(data["fault"]["x"], data["fault"]["y"])))
hf_x = np.array(data["hf"]["x"])
hf_y = np.array(data["hf"]["y"])
hf_mWm2 = np.array(data["hf"]["q"])

Plot the heat flow data and the anomaly:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_aspect('equal')
h = ax.scatter(hf_x, hf_y, c=hf_mWm2)
ax.plot(*fault_trace.T)
fig.colorbar(h)

### Regional Aggregate Heat Flow: Posterior Predictive
This part computes the posterior predictive heat flow distribution, i.e. the estimate
of the regional aggregate heat flow distribution.

*Note here that the above data set is truly gamma but a heat flow anomaly from the
fault has been superposed. The superposed heat flow anomaly stems from 90 MW heat
production over the length of the fault with a depth of 14 km.*

In [ ]:
gcp = default_prior()
predictive = HeatFlowPredictive(hf_mWm2, hf_x, hf_y, gcp, dmin=20e3)

qplt = np.linspace(30, 90)
cdf = predictive.cdf(qplt)

Some code from the artificially generated heat flow data. This would
not be available for a real-world data analysis:

In [ ]:
from scipy.special import gammainc
cdf_exact = gammainc(53.3, qplt)

Plot the results. Again, note that the synthetic data is truly gamma
distributed but overlayed by a 90 MW heat flow anomaly.

In [ ]:
fig = plt.figure(figsize=(6.2,4))
ax = fig.add_axes((0.09, 0.12, 0.89, 0.86))
ax.set_ylabel('CDF')
ax.set_xlim(30, 90)
ax.set_ylim(0,1.005)

ax.set_xlabel('Heat flow ($\\mathrm{mWm}^{-2}$)')
ax.step([0] + list(np.sort(hf_mWm2)), [0] + list((np.arange(hf_mWm2.size)+1)/hf_mWm2.size),
        where='post', linewidth=1.0, label='Empirical CDF')
qplt = np.linspace(30, 90)
ax.plot(qplt, cdf, linewidth=1.0, label="Posterior Predictive\nCDF")
ax.plot(qplt, cdf_exact, linestyle='--', linewidth=1.0, label="Original CDF")
ax.legend();

### Bayesian Heat Flow Anomaly Quantification
Now we quantify the heat flow anomaly that originates from the fault.
In this synthetic example, we know perfectly the geometry of the fault
and, indeed, the true anomaly signature $c_i$ at the data locations.

In [ ]:
anomaly = AnomalyLS1980(fault_trace, 14e3)
post = HeatFlowAnomalyPosterior(hf_mWm2, hf_x, hf_y, anomaly, gcp)

P_H = np.linspace(0, post.PHmax, 200)
pdf_P_H = post.pdf(P_H)
tail_P_H = post.tail(P_H)

In [ ]:
fig = plt.figure(figsize=(6.2,4))
ax = fig.add_axes((0.07, 0.12, 0.90, 0.84))
h0 = ax.plot(1e-6*P_H, pdf_P_H)
h1 = ax.axvline(90, linestyle='--', linewidth=1.0)
ax.set_ylim(0, ax.get_ylim()[1])
ax.set_xlim(0, 1e-6*post.PHmax)
ax.set_xlabel('Power $P_H$ (MW)')
ax.set_ylabel('Posterior density ($\mathrm{W}^{-1}$)')
ax2 = ax.twinx()
ax2.set_ylim(0,1)
h2 = ax2.plot(1e-6*P_H, tail_P_H, color='tab:orange')
ax.legend((h0[0], h2[0], h1), ('Posterior PDF', 'Posterior tail distribution', 'True Anomaly'));

### License
```
REHEATFUNQ Quickstart notebook.

This file is part of the REHEATFUNQ model.

Author: Malte J. Ziebarth (ziebarth@gfz-potsdam.de)

Copyright © 2022 Malte J. Ziebarth

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
```